# 📌 TF-IDF (비슷한 마이룸 추천)

## ✏️ DB 연결

In [119]:
import os
import pymysql
import pandas as pd
import re

from dotenv import load_dotenv, find_dotenv


# 환경변수 불러오기
load_dotenv(find_dotenv())
HOST = os.environ["MYSQL_HOST"] # MySQL host
DB = os.environ["MYSQL_NAME"]   # MySQL name
USER = os.environ["MYSQL_USER"] # MySQL user
PASSWORD = os.environ["MYSQL_PASS"] # MySQL password
PORT = int(os.environ["MYSQL_PORT"]) # MySQL port


# DB 연결
db = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWORD, db=DB, charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)
#  cursor생성
cursor = db.cursor()

In [120]:
# 한글, 숫자, 영문만 가져옴
def sub_special(s):
    # html 태그 제거
    tag_remover = re.compile('<.*?>')
    s = re.sub(tag_remover, '', s)
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

# 리스트를 문자열로
def list_2_str(list):
    return "".join(list)

### ✏️ 로그인 한 유저의 포트폴리오 Summary 및 Tag 조회

In [121]:
# 로그인 한 유저의 번호
user_seq = 40

# 포트폴리오 summary 조회
port_sql = "SELECT GROUP_CONCAT(DISTINCT(`p`.`summary`) SEPARATOR '') AS `portfolios` \
    FROM `portfolio` AS `p` \
    INNER JOIN `arrange` AS `a` ON `p`.`port_seq` = `a`.`port_seq` \
    INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
    INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
    WHERE `u`.`user_seq` = " + str(user_seq) + " GROUP BY `u`.`user_seq`; "

cursor.execute(port_sql)
result = cursor.fetchall()

ports = result[0].get("portfolios")

# Tag 조회
tag_sql = "SELECT GROUP_CONCAT(DISTINCT(`t`.`name`) SEPARATOR '') AS `tags` \
    FROM `tag` AS `t` \
    INNER JOIN `arrange` AS `a` ON `t`.`port_seq` = `a`.`port_seq` \
    INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
    INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
    WHERE `u`.`user_seq` = " + str(user_seq) + " GROUP BY `u`.`user_seq`; "

cursor.execute(tag_sql)
result = cursor.fetchall()
tags = result[0].get("tags")

user_feat = ports + tags
df = pd.DataFrame({"room_seq": [0], "feat": [user_feat]})
df

,room_seq,feat
0,0,<h1>아이링크</h1><p>스스로 모든 것을 하고 싶어하는 모든 아이를 위해 우리...


### 형태소 분석

In [122]:
from konlpy.tag import Okt
import pickle

okt = Okt()

file_path = "./stop_words.txt"
with open(file_path, encoding="utf8") as f:
    lines = f.readlines()
stop_words = [line.rstrip("\n") for line in lines]

path = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))
with open(path + "\\stop_words.pkl", "wb") as f:
    pickle.dump(stop_words, f, protocol=pickle.HIGHEST_PROTOCOL)


In [123]:
# 형태소 분석
def morph_and_stopword(text):
    text = sub_special(text)
    
    #형태소 분석
    words = okt.morphs(text, stem=True)
    
    # 형태소 분석 결과 담을 텍스트
    feat_list = []
    
    #불용어 처리
    for word in words:
        if word not in stop_words and len(word) > 1:
            feat_list.append(word)
            
    return list_2_str(set(feat_list))

---

### 📌 모든 방의 포트폴리오 Summary, Tag

In [124]:
# 포트폴리오 summary 조회
port_sql = "SELECT `r`.`room_seq`, GROUP_CONCAT(DISTINCT(`p`.`summary`) SEPARATOR '') AS `portfolios` \
    FROM `portfolio` AS `p` \
    INNER JOIN `arrange` AS `a` ON `p`.`port_seq` = `a`.`port_seq` \
    INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
    INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
    WHERE `r`.`user_seq` != " + str(user_seq) + " GROUP BY `r`.`room_seq` ;"

cursor.execute(port_sql)
result = cursor.fetchall()
df_ports = pd.DataFrame(result)

# Tag 조회
tag_sql = "SELECT `r`.`room_seq`, GROUP_CONCAT(DISTINCT(`t`.`name`) SEPARATOR '') AS `tags` \
    FROM `tag` AS `t` \
    INNER JOIN `arrange` AS `a` ON `t`.`port_seq` = `a`.`port_seq` \
    INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
    INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
    WHERE `r`.`user_seq` != " + str(user_seq) + " GROUP BY `r`.`room_seq` ;"

cursor.execute(tag_sql)
result = cursor.fetchall()
df_tags = pd.DataFrame(result)

# 태그 없을 시 빈 데이터 프레임 생성
if df_tags.empty:
    df_tags = pd.DataFrame({"room_seq": [], "tags": []})
    
# portfolio, tag 데이터 프레임 병합
df_rooms = pd.merge(df_ports, df_tags, how="outer")

# 결측값 제거
df_rooms["portfolios"] = df_rooms["portfolios"].fillna("")
df_rooms["tags"] = df_rooms["tags"].fillna("")

# 열 병합
df_rooms["feat"] = df_rooms["portfolios"] + df_rooms["tags"]
df_rooms.drop(["portfolios", "tags"], axis=1, inplace=True)

# 로그인 유저 데이터와 각 방의 데이터 프레임 병합
df = pd.concat([df, df_rooms], ignore_index=True)
df.drop_duplicates(inplace=True, ignore_index=True)

df["feat"] = df["feat"].map(lambda x:morph_and_stopword(x))
df

,room_seq,feat
0,0,dockerexpressnodejs전달만들어지다백엔드해보다기분부모님스스로태그일상링크...
1,82,fsdfdsfdsfsfSFAFSFSG


### ✏️ 방 번호:인덱스

In [125]:
seq_2_idx = dict(zip(df["room_seq"], df.index))

# dict의 value로 key 찾기
def get_key(val, dict):
    for key, value in dict.items():
        if val == value:
                return key

### TEST TF-IDF

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [127]:
tf_idf = TfidfVectorizer()
feat_list = list(df["feat"])
tf_idf_matrix = tf_idf.fit_transform(feat_list)
print("TF-IDF 행렬의 크기(shape): ", tf_idf_matrix.shape)
print(tf_idf_matrix)

cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
print("코사인 유사도 연산 결과: ",cosine_sim.shape)
print(cosine_sim)

TF-IDF 행렬의 크기(shape):  (2, 2)
  (0, 0)	1.0
  (1, 1)	1.0
코사인 유사도 연산 결과:  (2, 2)
[[1. 0.]
 [0. 1.]]


In [128]:
def get_recommendations(user_seq, cosine_sim=cosine_sim):
    
    idx = seq_2_idx[0]
    
    # 해당 데이터와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:9]
    
    # 추천 방 저장 리스트
    room_list = [idx[0] for idx in sim_scores]
    relation = list(df["room_seq"].iloc[room_list])
    print(relation)
    relation = ",".join(map(str, relation))
    sql = "INSERT INTO `relation` (user_seq, room_list) VALUES (" + str(user_seq) + ", '" + relation + "') \
        ON DUPLICATE KEY UPDATE user_seq = " + str(user_seq) + ";"
    cursor.execute(sql)
    db.close()
    
    return list(df["room_seq"].iloc[room_list])

In [129]:
get_recommendations(40)

[82]


[82]

[로그인 한 놈]  
- 모든 포트폴리오 목록 summary
- 모든 태그

[다른 사용자]  
- 방에 배치된 포트폴리오의 summary
- 해당 포트폴리오들의 태그  

유저-유저가 아닌 방-로그인 한 유저  

DataFrame  
[방 번호][포트폴리오내용][태그목록] 을
[방 번호][형태소분석 후의 feat str]로 만들어  
pickle로 저장 -> 데이터 로딩 시간 감소  



인덱스를 방 번호로 지정하고  
로그인한 유저의 텍스트랑  
방의 feat str의 유사도 계산  
유사도 순 정렬한 뒤 8개?  